In [8]:
import pandas as pd

# 파일 불러오기
obsr_df = pd.read_csv("기상/농업기상데이터_2018_2025.csv")
spot_info_df = pd.read_csv("기상/RDA_SPOT_INFO.csv")

# 지점명 병합
merged_df = obsr_df.merge(
    spot_info_df[['지점코드', '지점명']],
    how='left',
    left_on='obsr_Spot_Code',
    right_on='지점코드'
)

# 컬럼 순서 조정
merged_df = merged_df[['province', '지점명', 'obsr_Spot_Code', 'date', 'temp', 'rain', 'hum']]

# 컬럼명 RDA_SPOT_INFO 형식에 맞게 한글로 변경
merged_df.columns = ['도명', '지점명', '지점코드', '관측일자', '기온(°C)', '강수량(mm)', '상대습도(%)']

# 결과 저장 (옵션)
merged_df.to_csv("기상/농업기상데이터_지점명포함_컬럼정리.csv", index=False, encoding='cp949')

# 결과 확인
print(merged_df.head())

    도명      지점명        지점코드        관측일자  기온(°C)  강수량(mm)  상대습도(%)
0  경기도  가평군 가평읍  477802A001  2018-01-01    -4.2      0.0     62.0
1  경기도  가평군 가평읍  477802A001  2018-01-02    -4.0      0.0     58.1
2  경기도  가평군 가평읍  477802A001  2018-01-03    -5.6      0.0     58.2
3  경기도  가평군 가평읍  477802A001  2018-01-04    -6.3      0.0     68.3
4  경기도  가평군 가평읍  477802A001  2018-01-05    -4.6      0.0     68.7
